In [15]:
import gmaps
import pandas as pd
import requests
import json

# Google developer API key
from api_keys import g_key
from pprint import pprint

# Configure gmaps
gmaps.configure(api_key=g_key)

In [16]:
cw_df = pd.read_csv('output_data/cities.csv')
cw_df.head()

,City,Longitude,Latitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date Time
0,katsuura,140.3000,35.1333,39.31,45,31,19.39,JP,1643751097
1,puerto ayora,-90.3518,-0.7393,80.56,91,71,5.01,EC,1643751098
2,ancud,-73.8203,-41.8697,65.73,83,100,7.99,CL,1643751098
3,barcelos,-8.6151,41.5388,60.94,72,0,5.84,PT,1643751098
4,sao jose de ribamar,-44.0542,-2.5619,84.58,78,60,14.83,BR,1643751099


In [86]:
locs = cw_df[["Latitude", "Longitude"]]
humidity = cw_df["Humidity"]
fig = gmaps.figure(layout={'width': '900px', 'height': '600px'})
heat_layer = gmaps.heatmap_layer(locs, weights=humidity, 
                                 dissipating=False,
                                 point_radius=6)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='600px', width='900px'))

In [75]:
hotel_df = cw_df.loc[(cw_df['Max Temperature'] > 70) & (cw_df['Max Temperature'] < 80) \
                                          & (cw_df['Wind Speed'] < 10) \
                                          & (cw_df['Cloudiness'] < 10) \
                                          & (cw_df['Humidity'] < 50)].copy()

hotel_df.drop(columns = ['Max Temperature', 'Wind Speed', 'Cloudiness', 'Humidity', 'Date Time'], inplace = True)
hotel_df = hotel_df[['City', 'Country','Longitude','Latitude']]
hotel_df

,City,Country,Longitude,Latitude
119,vila velha,SN,-14.9500,12.8833
159,padang,MX,-109.9124,22.8909
204,lasa,MX,-100.5167,28.7000
258,uberaba,NG,3.8500,9.0833
317,geraldton,US,-97.5169,28.0170
329,pascagoula,ML,-3.9836,22.6783
619,portree,US,-99.7670,29.3669
643,boguchany,MX,-103.2500,25.5333


In [76]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location" : None,
    "radius" : 5000,
    "keyword" : "hotel",
    "key" : g_key
}


In [77]:
lats = []
lngs = []
hotels = []
for index, row in hotel_df.iterrows():
    params['location'] = f"{row['Latitude']},{row['Longitude']}"
    hotel = requests.get(base_url, params=params).json()
    try:
        lats.append(hotel['results'][0]['geometry']['location']['lat'])
        lngs.append(hotel['results'][0]['geometry']['location']['lng'])
        hotels.append(hotel['results'][0]['name'])
    except:
        hotel_df.drop(index, inplace = True)
        print(f"Missing field/result for {row['City'].title()}... dropping row.")


Missing field/result for Pascagoula... dropping row.
Missing field/result for Portree... dropping row.


In [78]:
hotel_df.drop(columns = ['Latitude', 'Longitude'], inplace = True)
hotel_df['Lat'] = lats
hotel_df['Lng'] = lngs
hotel_df['Hotel Name'] = hotels
hotel_df

,City,Country,Lat,Lng,Hotel Name
119,vila velha,SN,12.877297,-14.911203,Hôtel Relais De Kolda
159,padang,MX,22.890917,-109.907069,Medano Hotel and Suites
204,lasa,MX,28.687371,-100.558840,Hampton Inn by Hilton Piedras Negras
258,uberaba,NG,9.082926,3.836884,BASUAK HOTELS
317,geraldton,US,28.028582,-97.529505,Americas Best Value Inn Sinton
643,boguchany,MX,25.538512,-103.224974,HOTEL PLAZA MATAMOROS


In [87]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

markers = gmaps.marker_layer(coords, info_box_content=info_box)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='600px', width='900px'))